In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

include("data.jl")
using Flux, Statistics
using Flux: onehot, onehotbatch, throttle, crossentropy, reset!, onecold

const epochs = 20

train = gendata(100, 2)
val = gendata(10, 2)

scanner = LSTM(length(alphabet), 20)
encoder = Dense(20, length(alphabet))

function model(x)
    state = scanner.(x.data)[end]
    reset!(scanner)
    softmax(encoder(state))
end

loss(x, y) = crossentropy(model(x), y)
batch_loss(data) = mean(loss(d...) for d in data)

opt = ADAM(params(scanner, encoder))
evalcb = () -> @show batch_loss(val)

for i=1:epochs
    Flux.train!(loss, train, opt, cb=throttle(evalcb, 10))
end

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
batch_loss(val) = 0.6624844602398227 (tracked)
batch_loss(val) = 0.6407505889089489 (tracked)
batch_loss(val) = 0.6342658306804951 (tracked)
batch_loss(val) = 0.6310193242577828 (tracked)
batch_loss(val) = 0.6271901792243808 (tracked)
batch_loss(val) = 0.6185420803166053 (tracked)
batch_loss(val) = 0.5933091082566672 (tracked)
batch_loss(val) = 0.5253520432801917 (tracked)
batch_loss(val) = 0.4074461278583274 (tracked)
batch_loss(val) = 0.2764286361075904 (tracked)
batch_loss(val) = 0.16950319391141405 (tracked)
batch_loss(val) = 0.10237784032698341 (tracked)
batch_loss(val) = 0.06592752710060654 (tracked)
batch_loss(val) = 0.04566967877719173 (tracked)
batch_loss(val) = 0.03354856982984823 (tracked)
batch_loss(val) = 0.025748384580454496 (tracked)
batch_loss(val) = 0.020418843410227178 (tracked)
batch_loss(val) = 0.016602271569123205 (tracked)
batch_loss(val) = 0.0

sanity test

In [2]:
tx = map(c -> onehotbatch(c, alphabet), [
    [false, true], # 01 -> 1
    [true, false], # 10 -> 1
    [false, false], # 00 -> 0
    [true, true]]) # 11 -> 0
[onecold(model(x)) - 1 for x in tx] |> println

[1, 1, 0, 0]
